# Text-to-Video Generation with AnimateDiff

In [ ]:
# Download the necessary packages (globally)
!pip install torch diffusers opencv-python transformers accelerate peft

## Load Model into Memory

In [ ]:
import torch
from diffusers import AnimateDiffPipeline, DDIMScheduler, MotionAdapter
from diffusers.utils import export_to_gif, export_to_video

# Load the motion adapter
adapter = MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5-2", torch_dtype=torch.float16)
# load SD 1.5 based finetuned model
original_model = "emilianJR/CyberRealistic_V3"
# custom model path (for fine tuned models)
model_id = "rohan"

# Load the Model Weights into GPU Memory
pipe = AnimateDiffPipeline.from_pretrained(original_model, motion_adapter=adapter, torch_dtype=torch.float16)

## Motion Lora
We use motion loras for adding specific types of motions to the results. view more details at [Animated Diff](https://huggingface.co/docs/diffusers/api/pipelines/animatediff)

In [ ]:
# pipe.load_lora_weights(
#     "guoyww/animatediff-motion-lora-zoom-out", adapter_name="zoom-out",
# )
# pipe.load_lora_weights(
#     "guoyww/animatediff-motion-lora-pan-left", adapter_name="pan-left",
# )
# pipe.load_lora_weights(
#     "guoyww/animatediff-motion-lora-zoom-out", adapter_name="zoom-out",
# )
# pipe.load_lora_weights(
#     "guoyww/animatediff-motion-lora-tilt-up", adapter_name="tilt-up",
# )
pipe.load_lora_weights(
    "guoyww/animatediff-motion-lora-rolling-clockwise", adapter_name="rolling-clockwise",
)

# use if using multiple motion adapters
# pipe.set_adapters(["zoom-out", "pan-left"], adapter_weights=[1.0, 1.0])

scheduler = DDIMScheduler.from_pretrained(
    model_id,
    subfolder="scheduler",
    clip_sample=False,
    timestep_spacing="linspace",
    beta_schedule="linear",
    steps_offset=1,
)
pipe.scheduler = scheduler

# enable memory savings
pipe.enable_vae_slicing()
pipe.enable_model_cpu_offload()

## Inference

In [20]:
output = pipe(
    prompt="High quality, full-face centered closeup of zwx person as a casual yet stylish male, relaxed demeanor, sunlit beach backdrop, summer vibes, art by artgerm and greg rutkowski, warm and inviting color palette",
    negative_prompt="NSFW, amputee, bad anatomy, bad illustration, bad proportions, beyond the borders, blurry, body out of frame, boring background, branding, cropped, cut off, deformed, disfigured, dismembered, disproportioned, distorted, draft, duplicate, duplicated features, extra arms, extra fingers, extra hands, extra legs, extra limbs, fault, flaw, fused fingers, grains, grainy, gross proportions, improper scale, incorrect physiology, incorrect ratio, indistinct, logo, long neck, low quality, low resolution, multiple people, extra bodies, extra face, multiple faces, multiple frames, broken teeth",
    num_frames=16,
    guidance_scale=8,
    num_inference_steps=50,
    generator=torch.Generator("cpu").manual_seed(0),
)
frames = output.frames[0]
save_path = "rohan-13.gif"
export_to_gif(frames, save_path)

  0%|          | 0/50 [00:00<?, ?it/s]

'rohan-13.gif'

### Render Generation as GIF

In [21]:
from IPython.display import HTML
HTML(f'<img src="{save_path}" alt="GIF Image">')